<table width="100%" style="padding:0; margin-left:-6px;">
    <tr style="padding=0;" width="100%">
        <td width="auto">
           <hr style="border:2px solid darkblue">
            <h1> nb4. Query by scientific category</h1>
           <hr style="border:2px solid darkblue">
        </td>
        <td width="60px">
           <img src="ALMAsmall.png" align="right"/> 
        </td>
    </tr>
</table>    

The relevant columns in the ALMA TAP service are:
* *science_category* 
* *science_keyword*
 
--------- ----- -----

The scientific categories of observations in ALMA are the more general groups. These are:
* Active galaxies
* Cosmology
* Disks and planet formation
* Galaxy evolution
* ISM and star formation
* Local Universe
* Solar system
* Stars and stellar evolution
* Sun



In [1]:
import numpy as np
from astropy.table import Table
import pyvo
import sys
import matplotlib.pyplot as plt
import pandas as pd

service = pyvo.dal.TAPService("https://almascience.eso.org/tap")      # for the EU ALMA TAP service

# service = pyvo.dal.TAPService("https://almascience.nao.ac.jp/tap")  # for the EA ALMA TAP service
# service = pyvo.dal.TAPService("https://almascience.nrao.edu/tap")   # for the NA ALMA TAP service

<hr style="border:2px solid darkblue"> </hr>


## Query scientific category
<hr style="border:2px solid darkblue"> </hr>



In [2]:
def query_scientific_category(service, scientific_category):
    """Query for all observations of a given scientific category"""
    
    query = f"""  
            SELECT * 
            FROM ivoa.obscore  
            WHERE scientific_category = '{scientific_category}'  
            AND science_observation = 'T'  
            """

    return service.search(query).to_table().to_pandas()

<hr style="border:2px solid darkblue"> </hr>


## Query science keyword and data type
<hr style="border:2px solid darkblue"> </hr>




In [3]:
def query_science_keyword(service, science_keyword):
    """ALMA has a long list of scientific keywords in the Observing Tool from whch PIs need to select
       one or two in their proposals. This query returns all observations for a given science keyword (or part of it)"""
    
    query = f"""  
            SELECT *  
            FROM ivoa.obscore  
            WHERE science_observation = 'T'  
            AND science_keyword like '%{science_keyword}%'  
            """
    
    return service.search(query).to_table().to_pandas()

In [4]:
def query_science_keyword_datatype(service, science_keyword, datatype):
    """This function shows how to combine several constraints. Here the science keyword as well as the datatype (cube or image)"""
    
    query = f"""   
            SELECT *   
            FROM ivoa.obscore  
            WHERE science_keyword like '%{science_keyword}%'  
            AND science_observation = 'T'  
            AND dataproduct_type = '{datatype}'   
            """
    
    return service.search(query).to_table().to_pandas()

<hr style="border:2px solid darkblue"> </hr>


## Examples 

<hr style="border:2px solid darkblue"> </hr>




-------------------------------------------------

## Example 4a: How many active galaxies (science category) have been observed with ALMA? 
(Excluding calibrators)

In [5]:
output_agn = query_scientific_category(service, 'Active galaxies')

In [6]:
len(np.unique(output_agn['target_name']))

9790

List of AGN observed with ALMA:

In [7]:
np.set_printoptions(threshold=sys.maxsize)

### Show a few entries of the list
np.array(np.unique(output_agn['target_name']))[0:30]

array(['0-10000', '0-10510', '0-12043', '0-12407', '0-13299', '0-13375',
       '0-1426', '0-1437', '0-16822', '0-17244', '0-17749', '0-18038',
       '0-18180', '0-19883', '0-22825', '0-2318', '0-23382', '0-23626',
       '0-24625', '0-24636', '0-26339', '0-34302', '0-34622', '0-34897',
       '0-3662', '0-3753', '0-3973', '0-4356', '0-4503', '0-4936'],
      dtype=object)

In which bands have they been observed?

In [8]:
plt.rcParams["figure.figsize"] = (10,5)
output_agn['band_list'].hist(bins = 16)

<AxesSubplot:>

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x89 in position 0: invalid start byte

Where have they been observed?

In [9]:
plt.rcParams["figure.figsize"] = (20,15)
output_agn.plot(x='s_ra',y='s_dec', linestyle='', ms=7, marker='o', alpha=0.03, label='ALMA observed AGN')
plt.xlabel('RA')
plt.ylabel('DEC')


Text(0, 0.5, 'DEC')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x89 in position 0: invalid start byte

Within this *Science category* there are several combinations of *Science keywords*.

In [10]:
output_agn['science_keyword'].value_counts().plot.pie()

<AxesSubplot:ylabel='science_keyword'>

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x89 in position 0: invalid start byte

----------------------------

## Example 4b : How many sub-mm galaxies (science keyword) have been observed?

First, we investigate this question using the corresponding scientific keyboard.

In [11]:
output_smgs_scikey = query_science_keyword(service, 'Sub-mm Galaxies (SMG)')

print('The scientific keyword reports: ', len(output_smgs_scikey), 'observations with keyword "Sub-mm Galaxies (SMG)"')

The scientific keyword reports:  44206 observations with keyword "Sub-mm Galaxies (SMG)"


Second, we look for the keywords in the proposal abstracts.

In [12]:
query = """
        SELECT *
        FROM ivoa.obscore
        WHERE proposal_abstract like '%SMG%'
        OR proposal_abstract like '%sub-mm galax%'
        AND science_observation = 'T'
        """           

output_smgs_abs = service.search(query).to_table().to_pandas()

In [13]:
print('The query by abstract reports ', len(output_smgs_abs), 'observations of Sub-mm Galaxies (SMG)')

The query by abstract reports  25448 observations of Sub-mm Galaxies (SMG)


We can now plot the position of these sub-mm galaxies observations:

In [14]:
plt.rcParams["figure.figsize"] = (20,15)
output_smgs_scikey.plot(x='s_ra',y='s_dec', linestyle='', ms=5, marker='o', label='ALMA observed SMGs', alpha=0.01)

<AxesSubplot:xlabel='s_ra'>

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x89 in position 0: invalid start byte

We now search for the sources (coordinate positions)which have been observed multiple times and order them by number of observations

----------------------------

## Example 4c : Which sub-mm galaxies have been observed the most with ALMA?



To answer this question, we first group the observations by (almost) identical coordinates in order to obtain all observations for specific sources:

In [21]:
output_smgs_scikey['s_ra_round']  = np.round(output_smgs_scikey['s_ra'], 4) # since RA and DEC might deviate by 4 decimals ~ 1 arcsec
output_smgs_scikey['s_dec_round'] = np.round(output_smgs_scikey['s_dec'], 4)

### data frame grouped by coordinates
df_gb = output_smgs_scikey.groupby(['s_ra_round', 's_dec_round']).count().sort_values(by= 'target_name', ascending=False)

We now compute the total exposure time corresponding toeach of this sources (groups of observations:

In [22]:
target_names = []
exposure_times = []
for i in range(len(df_gb['target_name'])):
    target = np.unique(output_smgs_scikey.loc[(output_smgs_scikey['s_ra_round']==df_gb.index[i][0]) & (output_smgs_scikey['s_dec_round']==df_gb.index[i][1])]['target_name'])
    target_names.append(target)
    ### Get total exposure times by adding all exposure time entries within each target group.
    exptime=np.sum(output_smgs_scikey.loc[(output_smgs_scikey['s_ra_round']==df_gb.index[i][0]) & (output_smgs_scikey['s_dec_round']==df_gb.index[i][1])]['t_exptime'])
    exposure_times.append(exptime)

We then plot the 50 targets (SMGs) with the longest total exposure times:

In [23]:
nr = 50
inds = (np.array(exposure_times)*-1).argsort()

### Plot exposure times
plt.bar(np.arange(nr),np.array(exposure_times)[inds][0:nr]/3600, align='center', label='The most popular SMGs with ALMA')

### Plot source names
for i in range(0,nr):
    plt.text(i,(np.array(exposure_times)[inds][0:nr]/3600)[i]+2 , str(np.array(target_names)[inds][i]), color='black', fontweight='bold', rotation=90 )
plt.ylim(0,200)
plt.ylabel('Total exposure time [hrs]')
plt.legend(fontsize=20)

<ipython-input-23-26ffea94ab2a>:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  plt.text(i,(np.array(exposure_times)[inds][0:nr]/3600)[i]+2 , str(np.array(target_names)[inds][i]), color='black', fontweight='bold', rotation=90 )


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x89 in position 0: invalid start byte

----------------------------------------------

## Example 4d : How many quasars have been observed with ALMA?

### Spectral line observations

In [24]:
output_qsos_line = query_science_keyword_datatype(service, 'Quasars', 'cube' )

len(np.unique(output_qsos_line['target_name']))

649

### Continuum observations

In [27]:
output_qsos_line = query_science_keyword_datatype(service, 'Quasars', 'image' )

len(np.unique(output_qsos_line['target_name']))

823